<a href="https://colab.research.google.com/github/qrvdelossantos/CPE-311/blob/main/Hands_on_Activity_7_1_Data_Collection_and_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CPE311 Computational Thinking with Python**

Name: <br>
Date: <br>
Course & Section: <br>
Instructor: <br>

**Module 7**

**Data Wrangling with Pandas**

**7.1 Supplementay Activity**

**Exercise 1**<br>
We want to look at data for the Facebook, Apple, Amazon, Netflix, and Google (FAANG) stocks, but we were given each as a separate CSV file. Combine them into a single file and store the dataframe of the FAANG data as faang for the rest of the exercises:<br>

1. Read each file in.
2. Add a column to each dataframe, called ticker, indicating the ticker symbol it is for (Apple's is AAPL, for example). This is how you look up a stock. Each file's name is also the ticker symbol, so be sure to capitalize it.
3. Append them together into a single dataframe.
4. Save the result in a CSV file called faang.csv

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import os
import pandas as pd
path = "/content/drive/My Drive/Data"
files = ["aapl.csv", "amzn.csv", "fb.csv", "goog.csv", "nflx.csv"]
file_paths = [os.path.join(path, f) for f in files]

In [43]:
dfs = []
for file in file_paths:
  ticker = os.path.basename(file).split(".")[0].upper()
  df = pd.read_csv(file)
  df["ticker"] = ticker
  dfs.append(df)
faang = pd.concat(dfs, ignore_index=True)

faang.to_csv(os.path.join(path, "faang.csv"), index=False)

print("faang data save to faang.ccsv file")

faang data save to faang.ccsv file


**Exercise 2**<br>

          
1.With faang, use type conversion to change the date column into a
datetime and the volume column into integers. Then, sort by date and
ticker.<br>
2.Find the seven rows with the highest value for volume.<br>
3.Right now, the data is somewhere between long and wide format. Use melt() to make it completely long format.<br>
4.Hint: date and ticker are our ID variables (they uniquely identify each row).<br>
5.We need to melt the rest so that we don't have separate columns for open, high, low, close, and volume.<br>

In [44]:
import pandas as pd
#Rows for the highest value for volume
faang['date'] = pd.to_datetime(faang['date'])
faang['volume'] = faang['volume'].astype(int)
faang = faang.sort_values(by=['date', 'ticker'])

top7_volume = faang.nlargest(7, 'volume')
#Melt Process
faang_long = faang.melt(
    id_vars=['date', 'ticker'],
    value_vars=['open', 'high', 'low', 'close', 'volume'],
    var_name='variable',
    value_name='value'
)

print(top7_volume)
print(faang_long.head())

          date      open      high       low     close     volume ticker
644 2018-07-26  174.8900  180.1300  173.7500  176.2600  169803668     FB
555 2018-03-20  167.4700  170.2000  161.9500  168.1500  129851768     FB
559 2018-03-26  160.8200  161.1000  149.0200  160.0600  126116634     FB
556 2018-03-21  164.8000  173.4000  163.3000  169.3900  106598834     FB
182 2018-09-21  219.0727  219.6482  215.6097  215.9768   96246748   AAPL
245 2018-12-21  156.1901  157.4845  148.9909  150.0862   95744384   AAPL
212 2018-11-02  207.9295  211.9978  203.8414  205.8755   91328654   AAPL
        date ticker variable      value
0 2018-01-02   AAPL     open   166.9271
1 2018-01-02   AMZN     open  1172.0000
2 2018-01-02     FB     open   177.6800
3 2018-01-02   GOOG     open  1048.3400
4 2018-01-02   NFLX     open   196.1000


**Exercise 3**<br>
Using web scraping, search for the list of the hospitals, their
address and contact information. Save the list in a new csv file,
hospitals.csv.<br>
Using the generated hospitals.csv, convert the csv file into pandas
dataframe. Prepare the data using the necessary preprocessing
techniques.

In [45]:
import pandas as pd
import requests

WIKI_URL = "https://en.wikipedia.org/wiki/List_of_hospitals_in_the_Philippines"

def fetch_hospital_tables(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return pd.read_html(response.text)

def clean_and_filter_tables(tables):
    processed_frames = []
    for table in tables:
        columns = table.columns
        if "Name of Hospital" in columns and "Class" in columns:
            table = table.rename(columns={
                "Name of Hospital" : "Hospital Name",
                "Location": "Address"
            })
            table["Hospital Name"] = table["Hospital Name"].astype(str).str.strip()
            table["Address"] = table["Address"].astype(str).str.strip()
            table["Class"] = table["Class"].astype(str).str.strip()

            mask = table["Class"].str.contains(r"\b(LGU|DOH)\b", case=False, na=False)
            filtered = table.loc[mask, [
                "Hospital Name", "Address", "Class"
                ]]
            processed_frames.append(filtered)

    if not processed_frames:
        return pd.DataFrame(columns=["Hospital Name", "Address", "Class"])
    return pd.concat(processed_frames, ignore_index=True)

tables = fetch_hospital_tables(WIKI_URL)
hospital_df = clean_and_filter_tables(tables)

hospital_df = hospital_df.drop_duplicates()
hospital_df = hospital_df.head(100)

hospital_df.to_csv("hospitals.csv", index=False)

print(f"Total hospitals captured: {len(hospital_df)}")
display(hospital_df)

/tmp/ipython-input-1206049524.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(response.text)
/tmp/ipython-input-1206049524.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = table["Class"].str.contains(r"\b(LGU|DOH)\b", case=False, na=False)
/tmp/ipython-input-1206049524.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = table["Class"].str.contains(r"\b(LGU|DOH)\b", case=False, na=False)
/tmp/ipython-input-1206049524.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = table["Class"].str.contains(r"\b(LGU|DOH)\b", case=False, na=False)
/tmp/ipython-inp

Total hospitals captured: 100


/tmp/ipython-input-1206049524.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = table["Class"].str.contains(r"\b(LGU|DOH)\b", case=False, na=False)
/tmp/ipython-input-1206049524.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = table["Class"].str.contains(r"\b(LGU|DOH)\b", case=False, na=False)


,Hospital Name,Address,Class
0,Caloocan City Medical Center,"450 A. Mabini St., Caloocan City",LGU
1,Ospital ng Malabon,"F. Sevilla Boulevard, Tañong, Malabon City",LGU
2,San Lorenzo Ruiz General Hospital,"O. Reyes St., Rosita Subdivision, Santulan, Ma...",DOH Retained
3,Gat Andres Bonifacio Memorial Medical Center,"8001 Delpan St., Tondo, Manila",LGU
4,Ospital ng Tondo,"Jose Abad Santos Avenue, Tondo, Manila",LGU
...,...,...,...
95,Region II Trauma and Medical Center,"Magsaysay, Bayombong, Nueva Vizcaya",DOH Retained
96,Aurora Memorial Hospital,"San Luis Street, Poblacion, Baler",LGU
97,Jose C. Payumo Jr. Memorial Hospital,"San Ramon, Dinalupihan",LGU
98,Mariveles District Hospital,"Ave. of the Phils, Freeport Area of Bataan",LGU


**Conclusion**

I learned how to collect data from the web, like hospital details, using tools such as requests, BeautifulSoup, and pandas. I also learned how to clean and prepare the data by removing duplicates, filling missing values, and formatting text. These steps helped turn raw web information into a clear dataset that can be used for analysis.